In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# read data
gov_sorted = pd.read_excel('gov_sorted.xlsx', index_col=0)
soc_sorted = pd.read_excel('soc_sorted.xlsx', index_col=0)
env_sorted = pd.read_excel('env_sorted.xlsx', index_col=0)

factors = pd.read_excel('factors-NonFixedCap_NonFixedESG.xlsx', index_col=0)
factors = factors.join([gov_sorted, soc_sorted, env_sorted])

# missing information
factors = factors.dropna(how='any')

factors.head()

In [ ]:
#factors = factors.loc[factors.index > '2014-01-01']

In [ ]:
def estimate_mdl(fac, data, esg, name):
    betas = {}
    quantile_cols = list(filter(lambda x: re.match(pattern=f'{esg}_Q:\d+', string=x), data.columns))

    for i in quantile_cols:
        X = sm.add_constant(data[fac])
        Y = data[i]

        model = sm.OLS(endog=Y, exog=X)
        model = model.fit(cov_type='HAC', cov_kwds={'maxlags': 12})

        #display(model.summary())

        betas.update({i: {
            'mu': data[i].mean(),
            'beta': model.params['Mkt-RF'], 
            'alpha': model.params['const'], 
            't_stat (alpha=0)': model.tvalues['const'],
            'p_val (alpha=0)': model.pvalues['const'],
            'r2_adj': model.rsquared_adj
            }
        })

    betas = pd.DataFrame(betas).transpose()
    
    # fix indexing
    betas['Quantile'] = betas.index
    betas['Type'] = name
    betas['ESG'] = esg
    betas = betas.reset_index(drop=True)

    return betas


esg_out = estimate_mdl('Mkt-RF', factors, 'soc', 'CAPM')
esg_out = esg_out.append(estimate_mdl(['Mkt-RF', 'SMB', 'HML', 'MOM'], factors, 'soc', 'Carhart'))
esg_out = esg_out.append(estimate_mdl(['Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_soc', 'SUS_env', 'SUS_gov'], factors, 'soc', 'SUS'))

esg_out = esg_out.append(estimate_mdl('Mkt-RF', factors, 'env', 'CAPM'))
esg_out = esg_out.append(estimate_mdl(['Mkt-RF', 'SMB', 'HML', 'MOM'], factors, 'env', 'Carhart'))
esg_out = esg_out.append(estimate_mdl(['Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_soc', 'SUS_env', 'SUS_gov'], factors, 'env', 'SUS'))

esg_out = esg_out.append(estimate_mdl('Mkt-RF', factors, 'gov', 'CAPM'))
esg_out = esg_out.append(estimate_mdl(['Mkt-RF', 'SMB', 'HML', 'MOM'], factors, 'gov', 'Carhart'))
esg_out = esg_out.append(estimate_mdl(['Mkt-RF', 'SMB', 'HML', 'MOM', 'SUS_soc', 'SUS_env', 'SUS_gov'], factors, 'gov', 'SUS'))

esg_out

In [ ]:
esg_out.to_excel('portfolios_results.xlsx')

In [ ]:
plt.style.use('ggplot')

g = sns.FacetGrid(esg_out, col="Type", row='ESG', sharex=True, sharey=True, hue="Type", xlim=(0.65, 1.3), ylim=(0, 0.025))
g.fig.set_size_inches(15, 10)
g.fig.set_dpi(200)
g.map(sns.regplot, "beta", "mu", ci=None)

g.set_axis_labels(r"$\hat{\beta}_M$", r"$\mathbb{E}(r_{pf})$")
g.add_legend()

plt.show()